In [1]:
import numpy as np
import pandas as pd

In [6]:
import re

In [2]:
from pathlib import Path

CWD = Path.cwd()

In [28]:
# load data
df = pd.read_csv("data_leave_one_out.csv")
df.head()

,intent,query,Promotions,Card Promotions,Open Account,OCBC Singapore Account,OCBC Securities Account,OCBC Malaysia Account,NISP Account,Card Cancellation,...,Credit card application rejection,Rebates,How to redeem rewards,360 Account interest dispute,Statement Request,Passbook savings account statement,Credit card statement,Debit card statement,Investment account statement,Update details
0,Promotions,promotion -PRON- ?,0.054397,0.150561,0.852556,0.888380,0.853222,0.854053,0.825240,0.778786,...,0.754952,0.715897,0.787883,0.714959,0.865170,0.829989,0.798031,0.861068,0.824732,0.913722
1,Promotions,promotion available ?,0.398314,0.367402,0.791127,0.828506,0.799017,0.785015,0.720876,0.832902,...,0.701948,0.721365,0.823370,0.708754,0.737345,0.740767,0.786862,0.790735,0.819249,0.756076
2,Promotions,promotion,0.054397,0.150561,0.852556,0.888380,0.853222,0.854053,0.825240,0.778786,...,0.754952,0.715897,0.787883,0.714959,0.865170,0.829989,0.798031,0.861068,0.824732,0.913722
3,Promotions,-PRON- want promotion,0.109674,0.180680,0.707255,0.759581,0.753057,0.782716,0.656011,0.691250,...,0.652758,0.650996,0.753304,0.636704,0.703462,0.664049,0.675467,0.746608,0.727109,0.804883
4,Promotions,view promotion,0.310240,0.307525,0.701131,0.741588,0.707859,0.729585,0.671988,0.767357,...,0.684919,0.657583,0.736120,0.617589,0.731495,0.739908,0.643090,0.754707,0.742063,0.787432


In [29]:
# preprocessing
df = df.dropna().drop_duplicates()
df['query'] = df['query'].apply(lambda x: x.lower())
df['query'] = df['query'].apply(lambda x: re.sub(r'(!|\.|,|\(|\)|\[|\]|\\|\?|\$|#|%|\*)', '', x))

In [30]:
df.head(2)

,intent,query,Promotions,Card Promotions,Open Account,OCBC Singapore Account,OCBC Securities Account,OCBC Malaysia Account,NISP Account,Card Cancellation,...,Credit card application rejection,Rebates,How to redeem rewards,360 Account interest dispute,Statement Request,Passbook savings account statement,Credit card statement,Debit card statement,Investment account statement,Update details
0,Promotions,promotion -pron-,0.054397,0.150561,0.852556,0.888380,0.853222,0.854053,0.825240,0.778786,...,0.754952,0.715897,0.787883,0.714959,0.865170,0.829989,0.798031,0.861068,0.824732,0.913722
1,Promotions,promotion available,0.398314,0.367402,0.791127,0.828506,0.799017,0.785015,0.720876,0.832902,...,0.701948,0.721365,0.823370,0.708754,0.737345,0.740767,0.786862,0.790735,0.819249,0.756076


In [31]:
cluster_cols = list(df.columns.values)[2:]

In [37]:
def get_top_3_clusters(data, cluster_cols):
    data = data.copy()
    cluster_cols = cluster_cols.copy()

    data['clusters_top3'] = data.apply(lambda x: np.argsort(x[cluster_cols].values)[:3].tolist(), axis=1)

    intents = cluster_cols # get all tickers
    intent2index = {v: i for (i, v) in enumerate(intents)}

    data['target'] = data['intent'].apply(lambda x: intent2index[x])

    top_clusters_cols = pd.DataFrame(data['clusters_top3'].values.tolist(),columns = ['clusters_1','clusters_2','clusters_3']).reset_index(drop=True)
    data = data.reset_index(drop=True)
    data = pd.concat([data,top_clusters_cols], axis=1)

    data.drop(columns = 'clusters_top3', inplace=True)
    data.drop(columns = cluster_cols, inplace=True)
    
    # print(data.head())
    return data

In [42]:
def get_accuracy(data, top=1):
    data = data.copy()
    
    assert top in (1,2,3), "top must be in (0, 1, 2)"
    
    if top == 1:
        # top 1 accuracy
        accuracy = (data[(data['clusters_1'] == data['target'])].shape[0] / data.shape[0])
    elif top == 2:
        # top 2 accuracy
        data["exists"] = data.drop(data.columns[[0,1,2,5]], 1).isin(data["target"]).any(1)
        accuracy = sum(data['exists'])/ data.shape[0]
    elif top == 3:
        # top 3 accuracy
        data["exists"] = data.drop(data.columns[[0,1,2]], 1).isin(data["target"]).any(1)
        accuracy = sum(data['exists'])/ data.shape[0]
    else:
        raise ValueError("top must be in (0, 1, 2)") 
    
    print('Accuracy for top {} clustering result is {:.1%}'.format(top, accuracy))
    return accuracy

In [43]:
df2 = get_top_3_clusters(df, cluster_cols)
df2.head()

,intent,query,target,clusters_1,clusters_2,clusters_3
0,Promotions,promotion -pron-,0,0,1,11
1,Promotions,promotion available,0,1,0,27
2,Promotions,promotion,0,0,1,11
3,Promotions,-pron- want promotion,0,0,1,11
4,Promotions,view promotion,0,1,0,37


In [44]:
get_accuracy(df2, 1)
get_accuracy(df2, 2)
get_accuracy(df2, 3)

Accuracy for top 1 clustering result is 85.4%
Accuracy for top 2 clustering result is 94.7%
Accuracy for top 3 clustering result is 97.1%


0.9708994708994709

In [45]:
# load spacy
# load spacy model 
import spacy
nlp = spacy.load("en_core_web_sm")

from spacy.lang.en.stop_words import STOP_WORDS
stop_words = list(STOP_WORDS)

In [ ]:
def get_keywords(intent_list):
    """ Get list of keywords from intent """
    keywords = []
    for intent in list(set(intent_list)):
        keywords.extend(intent.strip().split(' '))
    keyword_list = list(set(keywords))
    keyword_list = [i.lower() for i in keyword_list if i.lower() not in stop_words]
    keyword_list.append('nsip')

    keyword_list_lemma = []
    text = nlp(' '.join([w for w in keyword_list]))
    for token in text:
        keyword_list_lemma.append(token.lemma_)
    return keyword_list_lemma

In [ ]:
keyword_list_lemma = get_keywords(intent_list)

In [ ]:
def get_nlp_features(df):
    """ Get keyword features from dataframe """
    data = df.copy()
    data['lemma'] = data['query'].apply(lambda x:' '.join([token.lemma_ for token in nlp(x) if token.lemma_ not in stop_words]))
    data['keyword'] = data['lemma'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.lemma_ in keyword_list_lemma])))

    data['noun'] = data['query'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.pos_ in ['NOUN','PROPN'] and token.lemma_ not in stop_words])))
    data['verb'] = data['query'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.pos_ in ['VERB'] and token.lemma_ not in stop_words])))
    data['adj'] = data['query'].apply(lambda x: list(set([token.lemma_ for token in nlp(x) if token.pos_ in ['ADJ'] and token.lemma_ not in stop_words])))

    data['noun'] = data['noun'].apply(lambda x: ' '.join([w for w in x]))
    data['verb'] = data['verb'].apply(lambda x: ' '.join([w for w in x]))
    data['adj'] = data['adj'].apply(lambda x: ' '.join([w for w in x]))
    data['keyword'] = data['keyword'].apply(lambda x: ' '.join([w for w in x]))
    return data